In [149]:
import duckdb
import pandas as pd
from fredapi import Fred
import certifi
import os
from datetime import datetime, timedelta, date
from config import FRED_API_KEY

os.environ['SSL_CERT_FILE'] = certifi.where() #API Key security requirements
fred = Fred(api_key=FRED_API_KEY)
today_date = date.today().strftime('%Y-%m-%d') #Fetch Date
# List of FRED series IDs
series_ids = ['GDP', 
              'GDPC1', 
              'CORESTICKM159SFRBATL', 
              'WM2NS', 
              'UNRATE', 
              'DFF', 
              'DGS10', 
              'PCE', 
              'INDPRO', 
              'MRTSSM44000USS',
              'PPIACO',
              'HOUST']

ModuleNotFoundError: No module named 'config'

In [ ]:
sql_query = ''' 
show tables
''' 

with duckdb.connect('data/fred') as con: 
    display(con.sql(sql_query).df())

#### Data Fetched from the Economic Research Federal Reserve Bank of St. Louis

In [ ]:
def fetch_series_data(series_id, start_date, end_date):
    """
    Fetch series data for a given series ID between start_date and end_date.
    
    Parameters:
    - series_id (str): The FRED series ID.
    - start_date (str): The start date for data retrieval.
    - end_date (str): The end date for data retrieval.
    
    Returns:
    - pd.Series: The time series data.
    """
    try:
        series_data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
        return series_data
    except Exception as e:
        print(f"Error fetching series data for {series_id}: {e}")
        return pd.Series([])

def fetch_metadata(series_id):
    """
    Fetch metadata for a given series ID.
    
    Parameters:
    - series_id (str): The FRED series ID.
    
    Returns:
    - dict: The metadata for the series.
    """
    try:
        metadata = fred.get_series_info(series_id)
        return metadata
    except Exception as e:
        print(f"Error fetching metadata for {series_id}: {e}")
        return {}

def construct_dataframe(series_data, metadata, series_id, fetch_date):
    """
    Construct a DataFrame from series data and metadata.
    
    Parameters:
    - series_data (pd.Series): The time series data.
    - metadata (dict): The metadata for the series.
    - series_id (str): The FRED series ID.
    - fetch_date (str): The date when the data was fetched.
    
    Returns:
    - pd.DataFrame: The constructed DataFrame with data and metadata.
    """
    try:
        df = pd.DataFrame(series_data, columns=['Index']).reset_index().rename(columns={'index': 'Date'})
        df['Name'] = metadata.get('title', '')
        df['SeriesID'] = series_id
        df['Source'] = f'https://fred.stlouisfed.org/series/{series_id}'
        df['Units'] = metadata.get('units', '')
        df['SeasonalAdjustment'] = metadata.get('seasonal_adjustment', '')
        df['Frequency'] = metadata.get('frequency', '')
        df['FREDLastUpdatedDate'] = metadata.get('last_updated', '')
        df['FetchDate'] = fetch_date
        return df
    except Exception as e:
        print(f"Error constructing dataframe for {series_id}: {e}")
        return pd.DataFrame()

def fetch_fred_data(series_ids):
    """
    Fetch data and metadata for a list of FRED series IDs.
    
    Parameters:
    - series_ids (list of str): The FRED series IDs.
    
    Returns:
    - list of pd.DataFrame: A list of DataFrames with data and metadata for each series.
    """
    today_date = date.today().strftime('%Y-%m-%d')
    twenty_years_ago = (date.today() - timedelta(days=20*365)).strftime('%Y-%m-%d')
    dataframes = []
    
    for series_id in series_ids:
        series_data = fetch_series_data(series_id, twenty_years_ago, today_date)
        if series_data.empty:
            continue
        metadata = fetch_metadata(series_id)
        df = construct_dataframe(series_data, metadata, series_id, today_date)
        dataframes.append(df)
    
    return dataframes


In [ ]:
# Fetch data
dataframes = fetch_fred_data(series_ids)

In [ ]:
# Display the fetched DataFrames
for df in dataframes:
    display(df)